In [32]:
import keyring

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

# Use paper trading, no real money used.
USE_PAPER = True

(api_key_type, secret_key_type) = ('api_key_paper', 'secret_key_paper') if USE_PAPER else ('api_key', 'secret_key')
api_key = keyring.get_password('alpaca', api_key_type)
secret_key = keyring.get_password('alpaca', secret_key_type)

if not api_key or not secret_key:
    raise Exception('Alpaca API key not found.')
trading_client = TradingClient(api_key, secret_key, paper=USE_PAPER)

# Get our account information.
account = trading_client.get_account()

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change}')

$199995.896225 is available as buying power.
Today's portfolio balance change: $0.0


In [33]:
# Search for AAPL
aapl_asset = trading_client.get_asset('AAPL')

if aapl_asset.tradable:
    print('We can trade AAPL.')

We can trade AAPL.


In [34]:
# Preparing market order
market_order_data = MarketOrderRequest(
                    symbol='AAPL',
                    qty=0.023,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )